In [42]:
import os
import vtk
import numpy as np

In [43]:
def calc_domain_size(stlBoundingBox):
    stlMinX,stlMaxX,stlMinY,stlMaxY,stlMinZ,stlMaxZ= stlBoundingBox
    bbX = stlMaxX - stlMinX
    bbY = stlMaxY - stlMinY
    bbZ = stlMaxZ - stlMinZ
    minX = stlMinX - 2*bbX
    maxX = stlMaxX + 7*bbX
    minY = stlMinY - 3*bbY
    maxY = stlMaxY + 3*bbY
    minZ = stlMinZ - 5*bbZ
    maxZ = stlMaxZ + 5*bbZ
    domain_size = (minX,maxX,minY,maxY,minZ,maxZ)
    return domain_size

    

In [44]:
# to calculate the max length of STL
def getMaxSTLDim(stlBoundingBox):
    stlMinX,stlMaxX,stlMinY,stlMaxY,stlMinZ,stlMaxZ= stlBoundingBox
    bbX = stlMaxX - stlMinX
    bbY = stlMaxY - stlMinY
    bbZ = stlMaxZ - stlMinZ
    return max(bbX,bbY,bbZ)

In [45]:
# to calculate the min size of stl
def getMinSTLDim(stlBoundingBox):
    stlMinX,stlMaxX,stlMinY,stlMaxY,stlMinZ,stlMaxZ= stlBoundingBox
    bbX = stlMaxX - stlMinX
    bbY = stlMaxY - stlMinY
    bbZ = stlMaxZ - stlMinZ
    return min(bbX,bbY,bbZ)

In [46]:
# to calculate nearest wall thickness for a target yPlus value
def calc_y(fluid_properties,L=1.0,u=1.0,target_yPlus=200):
    rho = fluid_properties['rho']
    nu = fluid_properties['nu']
    Re = u*L/nu
    Cf = 0.0592*Re**(-1./5.)
    tau = 0.5*rho*Cf*u**2.
    uStar = np.sqrt(tau/rho)
    y = target_yPlus*nu/uStar
    return y

In [47]:
fluid_properties = {'nu':1.0e-6,'rho':1.0e3}
y = calc_y(fluid_properties,u=2)

In [48]:
y

0.002480031197374939

In [49]:
# calculate nearest cell size for a given expansion ratio and layer count
def calc_cell_size(y_=0.001,nLayers=5,expRatio=1.2,thicknessRatio=0.3):
    max_y = y_*expRatio**(nLayers)
    return max_y*thicknessRatio

In [50]:
calc_cell_size(y)

0.0018513333687156018

In [51]:
def calc_refinement_levels(max_cell_size=0.1,target_cell_size=0.001):
    size_ratio = max_cell_size / target_cell_size
    n = np.log(size_ratio)/np.log(2.)
    #print(n)
    return int(np.ceil(n))


In [52]:
calc_refinement_levels(target_cell_size=0.00185)

6

In [53]:
def calc_nx_ny_nz(domain_size,target_cell_size):
    (minX,maxX,minY,maxY,minZ,maxZ) = domain_size
    nx = (maxX-minX)/target_cell_size
    ny = (maxY-minY)/target_cell_size
    nz = (maxZ-minZ)/target_cell_size
    nx, ny, nz = int(nx), int(ny), int(nz)
    return (nx,ny,nz)

In [54]:
domain_size = (0,40.0,-0.5,0.5,0,1)
calc_nx_ny_nz(domain_size,target_cell_size=0.01)

(4000, 100, 100)

In [55]:
def set_mesh_settings(stlBoundingBox,fluid_properties,U=1.0,maxCellSize=0.1):
    domain_size = calc_domain_size(stlBoundingBox=stlBoundingBox)
    maxSTLLength = getMaxSTLDim(stlBoundingBox)
    backgroundCellSize = min(maxSTLLength/4.,maxCellSize) # this is the size of largest blockMesh cells
    nx,ny,nz = calc_nx_ny_nz(domain_size,backgroundCellSize)
    L = maxSTLLength # this is the characteristic length to be used in Re calculations
    target_y = calc_y(fluid_properties,L,U,target_yPlus=200) # this is the thickness of closest cell
    targetCellSize = calc_cell_size(target_y,expRatio=1.3,thicknessRatio=0.4)
    refLevel = calc_refinement_levels(backgroundCellSize,targetCellSize)

    # print the summary of results
    print(f"Domain size {domain_size}")
    print(f"Simple grading: {nx},{ny},{nz}")
    print(f"Target Y:{target_y}")
    print(f"Refinement Level:{refLevel}")

    

In [56]:
stlBoundingBox = (0,10,0,1,0,1)
set_mesh_settings(stlBoundingBox,fluid_properties,maxCellSize=10)

Domain size (-20, 80, -3, 4, -5, 6)
Simple grading: 40,2,4
Target Y:0.00582618324777173
Refinement Level:9


In [59]:
# Function to read STL file and compute bounding box
def compute_bounding_box(stl_file_path):
    # Check if the file exists
    if not os.path.exists(stl_file_path):
        raise FileNotFoundError(f"File not found: {stl_file_path}. Make sure the file exists.")
    # Create a reader for the STL file
    reader = vtk.vtkSTLReader()
    reader.SetFileName(stl_file_path)
    reader.Update()

    # Get the output data from the reader
    poly_data = reader.GetOutput()
    #print(poly_data)

    # Calculate the bounding box
    bounds = poly_data.GetBounds()
    xmin, xmax, ymin, ymax, zmin, zmax = bounds

    #print("Bounding Box:")
    #print(f"  X range: {xmin} to {xmax}")
    #print(f"  Y range: {ymin} to {ymax}")
    #print(f"  Z range: {zmin} to {zmax}")

    # Optionally, return the bounding box as a tuple
    return bounds

In [61]:
bounds = compute_bounding_box("cad.stl")

Bounding Box:
  X range: -1.4500000476837158 to 0.5
  Y range: -1.5 to 1.5
  Z range: -3.0616171314629196e-17 to 2.5


In [62]:
bounds

(-1.4500000476837158, 0.5, -1.5, 1.5, -3.0616171314629196e-17, 2.5)